<a href="https://colab.research.google.com/github/SampMark/ETL-de-dados-da-PNP/blob/main/ETL_Portal_Dados_Abertos_IFRN___Tabela__Servidores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL para dados abertos do IFRN - tabela `servidores`

O objetivo deste código é criar um **pipeline de dados automatizado no modelo ETL (Extract, Transform, Load)** para transformar dados públicos brutos em um ambiente de alta performance para análise de dados:

1.  **Extrair (Extract)**: coletar de forma automática os dados públicos sobre os servidores do IFRN diretamente de seu portal de dados abertos.
2.  **Transformar (Transform)**: limpar, padronizar, enriquecer e modelar esses dados brutos, convertendo-os de um formato inconsistente para um conjunto de dados limpo, coeso e pronto para análise.
3.  **Carregar (Load)**: enviar o conjunto de dados já tratado para o **Google BigQuery**, um data warehouse na nuvem, que pode ser facilmente consultado, analisado e utilizado para criar painéis de _Business Intelligence_ (BI), relatórios e outras análises de dados.

O script executa uma sequência lógica e bem definida de etapas para atingir seu objetivo. O fluxo de trabalho é o seguinte:

## Fluxo de Trabalho

### **1. Autenticação e configuração inicial**:
O script começa autenticando o usuário no ambiente do Google Cloud. Este é um pré-requisito essencial para permitir que o código tenha permissão para gravar dados no BigQuery no final do processo.

### **2. Extração de metadados (_web scraping_)**:
* **Ação**: o código acessa a página web do dataset de servidores (`https://dados.ifrn.edu.br/dataset/servidores`) e utiliza a técnica de **Web Scraping** para extrair a data de "Última Atualização".
* **Propósito**: esta etapa é crucial para rastrear a "validade" ou "atualidade" dos dados. A data extraída é posteriormente adicionada a cada registro, permitindo saber a qual "fotografia" do tempo aquela informação pertence.
* **Tratamento**: a data extraída é formatada para o padrão `DD/MM/AAAA`.

#### **3. Extração dos Dados Principais**
* **Ação**: O script carrega o arquivo CSV contendo os dados brutos de todos os servidores diretamente da URL fornecida pelo portal de dados abertos.
* **Propósito**: Esta é a etapa principal de extração, onde o conjunto de dados a ser trabalhado é carregado em um **DataFrame** do pandas.

#### **4. Transformação e Limpeza dos Dados**
* Esta é a etapa mais extensa, onde a "mágica" da limpeza acontece. O código aplica uma série de tratamentos para padronizar e corrigir as inconsistências em várias colunas:
    * **`categoria`**: Padroniza os vínculos (ex: `docente` vira `Docente`).
    * **`campus`**: Converte siglas em nomes completos (ex: `CNAT` vira `Campus Natal - Central`), tornando os dados mais legíveis para o usuário final.
    * **`setor_suap`**: Remove a redundância do campus (ex: `DIAC/ZL` vira `DIAC`), o que permite agrupar os setores corretamente em análises.
    * **`funcao`**: Limpa a descrição da função, mantendo apenas o código principal (ex: `CD0004 - DIAD/ZN` vira `CD0004`).
    * **`jornada_trabalho`**: Simplifica as descrições (ex: `40 HORAS SEMANAIS` vira `40h Semanais`).
    * **`disciplina_ingresso`**: Um tratamento complexo que atribui "Não se Aplica" para servidores não docentes e limpa os valores para os docentes.

#### **5. Preparação do DataFrame Final para BI**
* **Ação**: O script seleciona apenas as colunas que foram tratadas e que são relevantes para a análise.
* **Propósito**: Descarta colunas desnecessárias (como URLs de fotos) e renomeia as colunas restantes para nomes mais descritivos e padronizados, ideais para um ambiente de BI (ex: `categoria_tratada` vira `Vinculo_de_Carreira`). Por fim, adiciona a data de atualização extraída na etapa 2 como uma nova coluna.

#### **6. Carga no BigQuery (Load)**
* **Ação**: O script se conecta ao projeto Google Cloud especificado e envia o DataFrame final (`df_servidores_bi`) para uma tabela no BigQuery.
* **Propósito**: Esta é a etapa final do pipeline. Ao usar a opção `if_exists='replace'`, o código garante que a tabela no BigQuery seja sempre substituída pela versão mais recente e tratada dos dados, mantendo o data warehouse sempre atualizado. Ele utiliza um esquema de tabela (`table_schema`) pré-definido para garantir a consistência dos tipos de dados.

In [1]:
# -*- coding: utf-8 -*-
"""
ETL Completo dos Dados de Servidores do IFRN com Carga no BigQuery.
"""

import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from google.colab import auth
from google.auth import default
import pandas_gbq

# --- Autenticação no Google Cloud (Necessário no Colab) ---
try:
    auth.authenticate_user()
    creds, project_id_default = default()
    pandas_gbq.context.credentials = creds
    pandas_gbq.context.project = project_id_default
    print("✔ Autenticação com o Google Cloud realizada com sucesso!")
except Exception as e:
    print(f"⚠️ Aviso: Falha na autenticação. A exportação para o BigQuery não funcionará. Erro: {e}")
print("-" * 50)

✔ Autenticação com o Google Cloud realizada com sucesso!
--------------------------------------------------


In [2]:
# @title --- 0. Extração da Data de Atualização (Web Scraping) ---
url_pagina = 'https://dados.ifrn.edu.br/dataset/servidores'
print(f"Extraindo a data de última atualização dos dados em {url_pagina}...")
last_updated_date = 'Data não extraída' # Valor padrão

try:
    response = requests.get(url_pagina)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    update_header = soup.find('th', string=re.compile(r'Última Atualização'))
    if update_header:
        update_date_cell = update_header.find_next_sibling('td')
        if update_date_cell:
            last_updated_date = update_date_cell.get_text(strip=True)
            print(f"Data de atualização da base de dados no site: {last_updated_date}")
except requests.exceptions.RequestException as e:
    print(f"Não foi possível extrair a data de atualização. Erro: {e}")

try:
    # 1) Localiza e separa as partes da data usando expressão regular.
    match = re.search(r'(\w+) (\d+), (\d+), (\d+:\d+) \((\w+)\)', last_updated_date)

    if match:
        # 2) Extrai os grupos da regex.
        month_name, day, year, time, timezone = match.groups()

        # 3) Tabela de conversão de nomes de meses (em PT-BR) para números (1–12).
        month_map = {
            'Janeiro': 1, 'Fevereiro': 2, 'Março': 3, 'Abril': 4,
            'Maio': 5, 'Junho': 6, 'Julho': 7, 'Agosto': 8,
            'Setembro': 9, 'Outubro': 10, 'Novembro': 11, 'Dezembro': 12
        }
        month_number = month_map.get(month_name)

        if month_number:
            # 4) Cria um objeto datetime apenas com ano, mês e dia.
            #    (A hora/minuto e o fuso foram ignorados de propósito.)
            date_obj = datetime(int(year), month_number, int(day))

            # 5) Formata a data no padrão DD/MM/AAAA.
            data = date_obj.strftime('%d/%m/%Y')
            print(f"Data formatada (DD/MM/AAAA): {data}")
        else:
            # Caso o nome do mês não esteja no dicionário acima.
            print(f"Não foi possível interpretar o mês: {month_name}")
            data = "Data Não Formatada"
    else:
        # Caso a string não obedeça ao formato esperado pela regex.
        print(f"Formato de data inesperado: {last_updated_date}")
        data = "Data Não Formatada"

except Exception as e:
    # Tratamento genérico de erros (por exemplo, valores inválidos).
    print(f"Ocorreu um erro durante a formatação da data: {e}")
    data = "Data Não Formatada"
print("-" * 50)

Extraindo a data de última atualização dos dados em https://dados.ifrn.edu.br/dataset/servidores...
Data de atualização da base de dados no site: Março 13, 2025, 10:49 (BRT)
Data formatada (DD/MM/AAAA): 13/03/2025
--------------------------------------------------


In [3]:
# @title --- 1. Carregamento dos Dados ---

# O mesmo link utilizado no seu notebook Colab.
url = 'https://dados.ifrn.edu.br/dataset/0c5c1c1a-7af8-4f24-ba37-a9eda0baddbb/resource/9adf1491-9e7c-4950-ab02-d3e12af37ec2/download/dados_extraidos_recursos_servidores.csv'
df = pd.read_csv(url, sep=';')

print("Dados originais carregados com sucesso.")
print(f"O DataFrame possui {df.shape[0]} linhas e {df.shape[1]} colunas.")
print("-" * 50)

display(df.head())

Dados originais carregados com sucesso.
O DataFrame possui 2972 linhas e 13 colunas.
--------------------------------------------------


,categoria,cargo,setor_siape,disciplina_ingresso,setor_suap,nome,funcao,jornada_trabalho,telefones_institucionais,matricula,curriculo_lattes,campus,url_foto_75x100
0,docente,PROFESSOR ENS BASICO TECN TECNOLOGICO,DIAC/ZL,Didática,DIAC/ZL,Abigail Noadia Barbalho da Silva,FAG-IFRN - UAB/IFRN / FAG-IFRN - CESPEP/ZL,DEDICACAO EXCLUSIVA,(84) 3092-8915 (ramal: 8915),1895370,NaN,ZL,/media/fotos/75x100/98189.F0xg7aLNxBHA.jpg
1,docente,PROF ENS BAS TEC TECNOLOGICO-SUBSTITUTO,DIAC/MO,NaN,DIAC/MO,Abimael Esdras Carvalho de Moura Lira,NaN,40 HORAS SEMANAIS,(84) 3422-2669 (ramal: 2669),3268131,NaN,MO,/media/fotos/75x100/387388.o9YeUaXaDOSo.jpg
2,tecnico_administrativo,ADMINISTRADOR,DIAD/ZN,-,DIAD/ZN,Abinoam Soares da Silva,CD0004 - DIAD/ZN,40 HORAS SEMANAIS,(84) 4006-9502 (ramal: 9502),1845427,NaN,ZN,/media/fotos/75x100/9698.fk2bWDbZK29Q.jpg
3,docente,PROFESSOR ENS BASICO TECN TECNOLOGICO,DIAC/MO,Instalações Prediais,DIAC/MO,Abraao Jhonny da Costa Brazao,NaN,DEDICACAO EXCLUSIVA,(84) 3422-2669 (ramal: 2669),1266198,NaN,MO,/media/fotos/75x100/274524.CADei8Dy0ygg.jpg
4,docente,PROFESSOR ENS BASICO TECN TECNOLOGICO,DIAC/JC,Arte - Teatro,DIAC/JC,Abraao Lincoln Rosendo Frazao,NaN,DEDICACAO EXCLUSIVA,ramal: 5604 / ramal: 5629 / ramal: 5625,2460058,NaN,JC,/media/fotos/75x100/170241.0uazsCmttZVZ.jpg


In [4]:
df['matricula'] = df['matricula'].astype('object')
df['curriculo_lattes'] = df['curriculo_lattes'].astype('object')

display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2972 entries, 0 to 2971
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   categoria                 2972 non-null   object
 1   cargo                     2866 non-null   object
 2   setor_siape               2970 non-null   object
 3   disciplina_ingresso       2788 non-null   object
 4   setor_suap                2971 non-null   object
 5   nome                      2972 non-null   object
 6   funcao                    735 non-null    object
 7   jornada_trabalho          2972 non-null   object
 8   telefones_institucionais  2781 non-null   object
 9   matricula                 2972 non-null   object
 10  curriculo_lattes          0 non-null      object
 11  campus                    2972 non-null   object
 12  url_foto_75x100           2972 non-null   object
dtypes: object(13)
memory usage: 302.0+ KB


None

In [5]:
# @title --- 2. Visualização de valores únicos das colunas especificadas ---

columns_to_display = ['categoria', 'cargo', 'disciplina_ingresso', 'setor_suap', 'funcao', 'jornada_trabalho', 'campus']

for column in columns_to_display:
    unique_values = df[column].unique()
    # Convert all values to string before sorting to handle NaN
    unique_values_as_strings = [str(value) for value in unique_values]
    sorted_unique_values = sorted(unique_values_as_strings)
    print(f"\nTotal de valores exclusivos para '{column}': {len(unique_values_as_strings)}")
    print(f"Exibindo valores exclusivos para coluna '{column}':")
    for value in sorted_unique_values:
        print(value)
    print("-" * 30)


Total de valores exclusivos para 'categoria': 4
Exibindo valores exclusivos para coluna 'categoria':
docente
estagiario
indefinida
tecnico_administrativo
------------------------------

Total de valores exclusivos para 'cargo': 88
Exibindo valores exclusivos para coluna 'cargo':
ADMINISTRADOR
ANALISTA DE TEC DA INFORMACAO
ARQUITETO E URBANISTA
ARQUIVISTA
ASSISTENTE DE ALUNO
ASSISTENTE DE LABORATORIO
ASSISTENTE EM ADMINISTRACAO
ASSISTENTE SOCIAL
AUDITOR
AUX EM ADMINISTRACAO
AUXILIAR DE ARTES GRAFICAS
AUXILIAR DE BIBLIOTECA
AUXILIAR DE ELETRICISTA
AUXILIAR DE ENFERMAGEM
AUXILIAR DE LABORATORIO
AUXILIAR DE MECANICA
AUXILIAR EM ASSUNTOS EDUCACIONAIS
BIBLIOTECARIO-DOCUMENTALISTA
CARPINTEIRO
CONTADOR
DESENHISTA TECNICO ESPECIALIDADE
DESENHISTA-PROJETISTA
DIAGRAMADOR
ECONOMISTA
ELETRICISTA
ENFERMEIRO DO TRABALHO
ENFERMEIRO-AREA
ENGENHEIRO AGRONOMO
ENGENHEIRO DE SEG DO TRABALHO
ENGENHEIRO-AREA
ESTAGIARIO DE NIVEL SUPERIOR
ESTAGIARIO JORNADA 30 HORAS
ESTAGIARIO NIVEL MEDIO - INPE
ESTATISTICO
F

In [6]:
# @title --- 3. Tratamento dos parâmentos de colunas de interesse ---
# --- 3.1. Tratamento da Coluna 'categoria' ---
# Para melhor visualização em ferramentas de BI, vamos padronizar os textos.
print("Tratando a coluna 'categoria'...")
mapeamento_categoria = {
    'docente': 'Docente',
    'tecnico_administrativo': 'Técnico Administrativo',
    'estagiario': 'Estagiário',
    'indefinida': 'Não Definida' # Garante que valores indefinidos sejam claros.
}
df['categoria_tratada'] = df['categoria'].map(mapeamento_categoria).fillna('Outros')
print("Valores únicos em 'categoria_tratada':")
print(df['categoria_tratada'].unique())
print("-" * 50)

# --- 3.2. Tratamento da Coluna 'campus' ---
# Substituir as siglas por nomes completos torna os filtros e gráficos no BI mais intuitivos.
print("Tratando a coluna 'campus'...")
mapeamento_campus = {
    'AP': 'Apodi',
    'CA': 'Caicó',
    'CAL': 'Natal - Cidade Alta',
    'CANG': 'Canguaretama',
    'CM': 'Ceará-Mirim',
    'CN': 'Currais Novos',
    'CNAT': 'Natal - Central',
    'IP': 'Ipanguaçu',
    'JC': 'João Câmara',
    'JUC': 'Jucurutu',
    'LAJ': 'Lajes',
    'MC': 'Macau',
    'MO': 'Mossoró',
    'NC': 'Nova Cruz',
    'PAAS': 'Parelhas',
    'PAR': 'Parnamirim',
    'PF': 'Pau dos Ferros',
    'RE': 'Reitoria',
    'SC': 'Santa Cruz',
    'SGA': 'São Gonçalo do Amarante',
    'SPP': 'São Paulo do Potengi',
    'ZL': 'Natal - Zona Leste',
    'ZN': 'Natal - Zona Norte'
}
df['campus_tratado'] = df['campus'].map(mapeamento_campus).fillna('Não Identificado')
print("Valores únicos em 'campus_tratado':")
# Mostra os valores únicos ordenados para fácil verificação
print(sorted(df['campus_tratado'].unique()))
print("-" * 50)

# --- 3.3. Tratamento da Coluna 'setor_suap' (Aperfeiçoado) ---
# Implementando a lógica sugerida para remover a redundância do campus.
print("Tratando a coluna 'setor_suap' com a lógica aperfeiçoada...")
print(f"Valores únicos antes do tratamento: {df['setor_suap'].nunique()}")

# Usamos .astype(str) para evitar erros com valores nulos (NaN)
# e aplicamos a sua lógica para dividir a string e pegar a primeira parte.
df['setor_tratado'] = df['setor_suap'].astype(str).str.split('/').str[0]

# Substituímos o valor 'nan' (que vem de células vazias) por um texto claro.
df['setor_tratado'] = df['setor_tratado'].replace('nan', 'Setor Não Informado')

print(f"Valores únicos após o tratamento: {df['setor_tratado'].nunique()}")
print("Exemplo de transformação:")
print(df[['setor_suap', 'setor_tratado']].head())
print("-" * 50)

# --- 3.4. Tratamento da Coluna 'funcao' ---
# Removendo a descrição detalhada da função para manter apenas o código/título.
print("Tratando a coluna 'funcao'...")
df['funcao_tratado'] = df['funcao'].astype(str).str.split(' - ').str[0]
# Código corrigido para evitar o FutureWarning
df['funcao_tratado'] = df['funcao_tratado'].replace('nan', 'Sem Função')
print("Exemplo de transformação da coluna 'funcao':")
# Mostra as linhas que de fato possuem uma função para um melhor exemplo
print(df[df['funcao'].notna()][['funcao', 'funcao_tratado']].head())
print("-" * 50)

# --- 3.5. Tratamento da Coluna 'jornada_trabalho' ---
print("Tratando a coluna 'jornada_trabalho'...")
mapeamento_jornada = {
    '20 HORAS SEMANAIS': '20h Semanais',
    '25 HORAS SEMANAIS': '25h Semanais',
    '30 HORAS SEMANAIS': '30h Semanais',
    '40 HORAS SEMANAIS': '40h Semanais',
    'DEDICACAO EXCLUSIVA': 'Dedicação Exclusiva'
}
df['jornada_tratada'] = df['jornada_trabalho'].map(mapeamento_jornada).fillna('Não Informada')
print("Exemplo de transformação da coluna 'jornada_trabalho':")
print(df[['jornada_trabalho', 'jornada_tratada']].head())
print("-" * 50)

# --- 3.6. Tratamento da Coluna 'disciplina_ingresso' ---
print("Tratando a coluna 'disciplina_ingresso'...")
# Primeiro, preenchemos os valores nulos (NaN) e '-' com uma string única para facilitar a substituição.
df['disciplina_tratada'] = df['disciplina_ingresso'].fillna('N/A').replace('-', 'N/A')

# Para servidores que não são docentes, a disciplina de ingresso não se aplica.
# Usamos np.where para aplicar essa lógica condicional.
df['disciplina_tratada'] = np.where(
    df['categoria_tratada'] != 'Docente',
    'Não se Aplica',
    df['disciplina_tratada']
)

# Para docentes que possam ter ficado com 'N/A', trocamos por um texto mais claro.
df['disciplina_tratada'] = df['disciplina_tratada'].replace('N/A', 'Disciplina Não Informada')

print("Exemplo de transformação da coluna 'disciplina_ingresso':")
print(df[['categoria_tratada', 'disciplina_ingresso', 'disciplina_tratada']].head(10))
print("-" * 50)

# --- 3.7. Função para padronização da coluna 'Cargos' ---
def padronizar_cargos(df_column):
    """
    Padroniza os nomes dos cargos para agrupar funções semelhantes e corrigir inconsistências.
    """

    # 1. Agrupamento inicial com regex
    mapeamento_cargos = {
        # Agrupamento de Professores
        r'.*PROFESSOR.*': 'Professor do EBTT',
        r'.*PROF.*': 'Professor do EBTT',
        # 'PROF DO ENSINO BASICO TEC TECNOLOGICO', 'PROF ENS BAS TEC TECNOLOGICO - VISITANTE',
        # 'PROF ENS BAS TEC TECNOLOGICO-SUBSTITUTO', 'PROFESSOR ENS BASICO TECN TECNOLOGICO'

        # Padronização de Cargos Técnicos e Administrativos (removendo sufixos)
        #'TECNICO DE LABORATORIO AREA': 'Técnico de Laboratório',
        'DESENHISTA-PROJETISTA': 'Desenhista Projetista',
        'TEC EM SEGURANCA DO TRABALHO': 'Técnico em Segurança do Trabalho',
        'TEC EM METROLOGIA E QUALIDADE': 'Técnico em Metrologia e Qualidade',
        'ENGENHEIRO DE SEG DO TRABALHO': 'Engenheiro de Segurança do Trabalho',
        'OPERADOR DE MAQ DE LAVANDERIA': 'Operador de Máquina de Lavanderia',
        # 'TECNOLOGO-FORMACAO': 'Tecnólogo',
        'NUTRICIONISTA-HABILITACAO': 'Nutricionista/habilitação',
        'BIBLIOTECARIO-DOCUMENTALISTA': 'Bibliotecário',
        'ANALISTA DE TEC DA INFORMACAO': 'Analista de Tecnologia da Informação',
        'TEC DE TECNOLOGIA DA INFORMACAO': 'Técnico de Tecnologia da Informação',
        'ODONTOLOGO - 30H - DEC JUD': 'Odontólogo',
        'ODONTOLOGO - 40 HORAS': 'Odontólogo',

        # Agrupamento de Estagiários
        r'.*ESTAGIARIO.*': 'Estagiário',
        # 'ESTAGIARIO DE NIVEL SUPERIOR', 'ESTAGIARIO JORNADA 30 HORAS', 'ESTAGIARIO NIVEL MEDIO - INPE'

        # Correções pontuais
        'AUX EM ADMINISTRACAO': 'Auxiliar em Administração',
        # 'ASSISTENTE EM ADMINISTRACAO': 'Assistente em Administração',
        'TRADUTOR INTERPRETE DE LINGUAGEM SINAIS': 'Tradutor e Intérprete de Linguagem de Sinais'
    }

    # Aplica o mapeamento com regex
    cargo_tratado = df_column.astype(str).replace(mapeamento_cargos, regex=True)

    # Remove o sufixo '-AREA'
    cargo_tratado = cargo_tratado.str.replace('-AREA', '/área', regex=False)
    cargo_tratado = cargo_tratado.str.replace(' AREA', '/área', regex=False)
    cargo_tratado = cargo_tratado.str.replace('-FORMACAO', '/formação', regex=False)

    # Converte para o formato Título (Primeira Letra Maiúscula) e remove espaços extras
    cargo_tratado = cargo_tratado.str.title().str.strip()

        # 3. Mapeamento para acentuação e correções finais
    mapeamento_acentos = {
        'Administracao': 'Administração',
        'Agropecuaria': 'Agropecuária',
        'Agronomo': 'Agrônomo',
        'Edificacoes': 'Edificações',
        'Eletrotecnica': 'Eletrotécnica',
        'Estatistico': 'Estatístico',
        'Fisico': 'Físico',
        'Graficas': 'Gráficas',
        'Laboratorio': 'Laboratório',
        'Laticinios': 'Laticínios',
        'Mecanica': 'Mecânica',
        'Mecanico': 'Mecânico',
        'Medico': 'Médico',
        'Mineracao': 'Mineração',
        'Veterinario': 'Veterinário',
        'Publicitario': 'Publicitário',
        'Psicologo': 'Psicólogo',
        'Quimica': 'Química',
        'Tecnico': 'Técnico'
    }

    # Aplica as substituições de acentuação
    for termo, correto in mapeamento_acentos.items():
        cargo_tratado = cargo_tratado.str.replace(termo, correto, regex=False)

    # Padroniza conectivos em minúsculas após a capitalização
    cargo_tratado = cargo_tratado.str.replace(r'\b(Da|Do|De|E|Em)\b', lambda m: m.group(1).lower(), regex=True)

    # Trata valores nulos ou vazios que restaram
    cargo_tratado = cargo_tratado.replace(['Nan', ''], 'Não Informado')

    return cargo_tratado

# Aplica a função de padronização de cargos
df['cargo_tratado'] = padronizar_cargos(df['cargo'])

print("Exemplo de transformação da coluna 'cargo':")
print(df[['cargo', 'cargo_tratado']].head(10))
print("-" * 50)

# Inclui a coluna 'Ano' a partir de 'last_updated_date'
df['Ano'] = data # Corrigido para usar a variável 'data' formatada

Tratando a coluna 'categoria'...
Valores únicos em 'categoria_tratada':
['Docente' 'Técnico Administrativo' 'Estagiário' 'Não Definida']
--------------------------------------------------
Tratando a coluna 'campus'...
Valores únicos em 'campus_tratado':
['Apodi', 'Caicó', 'Canguaretama', 'Ceará-Mirim', 'Currais Novos', 'Ipanguaçu', 'João Câmara', 'Jucurutu', 'Lajes', 'Macau', 'Mossoró', 'Natal - Central', 'Natal - Cidade Alta', 'Natal - Zona Leste', 'Natal - Zona Norte', 'Nova Cruz', 'Parelhas', 'Parnamirim', 'Pau dos Ferros', 'Reitoria', 'Santa Cruz', 'São Gonçalo do Amarante', 'São Paulo do Potengi']
--------------------------------------------------
Tratando a coluna 'setor_suap' com a lógica aperfeiçoada...
Valores únicos antes do tratamento: 578
Valores únicos após o tratamento: 225
Exemplo de transformação:
  setor_suap setor_tratado
0    DIAC/ZL          DIAC
1    DIAC/MO          DIAC
2    DIAD/ZN          DIAD
3    DIAC/MO          DIAC
4    DIAC/JC          DIAC
-------------

In [7]:
unique_cargo_values = df['cargo_tratado'].unique()
sorted_unique_cargo_values = sorted([str(value) for value in unique_cargo_values])

print(f"\nTotal de valores exclusivos para 'cargo_tratado': {len(sorted_unique_cargo_values)}")
print("Exibindo valores exclusivos para coluna 'cargo_tratado' em ordem alfabética:")
for value in sorted_unique_cargo_values:
    print(value)
print("-" * 30)


Total de valores exclusivos para 'cargo_tratado': 82
Exibindo valores exclusivos para coluna 'cargo_tratado' em ordem alfabética:
Administrador
Analista de Tecnologia da Informação
Arquiteto e Urbanista
Arquivista
Assistente Social
Assistente de Aluno
Assistente de Laboratório
Assistente em Administração
Auditor
Auxiliar de Artes Gráficas
Auxiliar de Biblioteca
Auxiliar de Eletricista
Auxiliar de Enfermagem
Auxiliar de Laboratório
Auxiliar de Mecânica
Auxiliar em Administração
Auxiliar em Assuntos Educacionais
Bibliotecário
Carpinteiro
Contador
Desenhista Projetista
Desenhista Técnico Especialidade
Diagramador
Economista
Eletricista
Enfermeiro do Trabalho
Enfermeiro/Área
Engenheiro Agrônomo
Engenheiro de Segurança do Trabalho
Engenheiro/Área
Estagiário
Estatístico
Fisioterapeuta
Físico
Impressor
Jornalista
Marceneiro
Mecânico
Motorista
Médico Veterinário
Médico/Área
Nutricionista/Habilitação
Não Informado
Odontólogo
Operador de Maquina Copiadora
Operador de Máquina de Lavanderia
Opera

In [8]:
columns_to_display = ['setor_tratado']

for column in columns_to_display:
    unique_values_count = df[column].value_counts()
    print(f"Contagem de valores exclusivos para coluna '{column}':")
    display(unique_values_count)
    print("-" * 30)

Contagem de valores exclusivos para coluna 'setor_tratado':


,count
setor_tratado,
DIAC,1343
COAES,134
COLAB,79
COAPAC,70
SEAC,59
...,...
COORD_MEC,1
DEAD,1
COMEC,1


------------------------------


In [9]:
columns_to_display = ['funcao_tratado']

for column in columns_to_display:
    unique_values_count = df[column].value_counts()
    print(f"Contagem de valores exclusivos para coluna '{column}':")
    display(unique_values_count)
    print("-" * 30)

Contagem de valores exclusivos para coluna 'funcao_tratado':


,count
funcao_tratado,
Sem Função,2237
FG0002,148
FUC0001,123
FG0001,110
SUB-CHEFIA,105
FAG-IFRN,95
CD0004,59
CD0003,23
CD0002,22


------------------------------


In [10]:
# --- 3. Limpeza e Seleção de Colunas ---
print("Selecionando e renomeando colunas para o DataFrame final...")
colunas_uteis = [
    'nome', 'matricula', 'categoria_tratada', 'campus_tratado',
    'setor_tratado', 'funcao_tratado', 'cargo_tratado',
    'jornada_tratada', 'disciplina_tratada'
]
df_servidores_bi = df[colunas_uteis].copy()

df_servidores_bi.rename(columns={
    'nome': 'Nome_do_Servidor',
    'matricula': 'Matricula',
    'categoria_tratada': 'Vinculo_de_Carreira',
    'campus_tratado': 'Campus_do_IFRN',
    'setor_tratado': 'Setor',
    'funcao_tratado': 'Funcao',
    'cargo_tratado': 'Cargo',
    'jornada_tratada': 'Jornada_de_Trabalho',
    'disciplina_tratada': 'Disciplina_de_Ingresso'
}, inplace=True)

df_servidores_bi['Ano'] = data
df['Ano'] = pd.to_datetime(df['Ano'], errors='coerce')

display(df_servidores_bi.head())
print("-" * 50)


Selecionando e renomeando colunas para o DataFrame final...


/tmp/ipython-input-1070266850.py:23: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Ano'] = pd.to_datetime(df['Ano'], errors='coerce')


,Nome_do_Servidor,Matricula,Vinculo_de_Carreira,Campus_do_IFRN,Setor,Funcao,Cargo,Jornada_de_Trabalho,Disciplina_de_Ingresso,Ano
0,Abigail Noadia Barbalho da Silva,1895370,Docente,Natal - Zona Leste,DIAC,FAG-IFRN,Professor do Ebtt,Dedicação Exclusiva,Didática,13/03/2025
1,Abimael Esdras Carvalho de Moura Lira,3268131,Docente,Mossoró,DIAC,Sem Função,Professor do Ebtt,40h Semanais,Disciplina Não Informada,13/03/2025
2,Abinoam Soares da Silva,1845427,Técnico Administrativo,Natal - Zona Norte,DIAD,CD0004,Administrador,40h Semanais,Não se Aplica,13/03/2025
3,Abraao Jhonny da Costa Brazao,1266198,Docente,Mossoró,DIAC,Sem Função,Professor do Ebtt,Dedicação Exclusiva,Instalações Prediais,13/03/2025
4,Abraao Lincoln Rosendo Frazao,2460058,Docente,João Câmara,DIAC,Sem Função,Professor do Ebtt,Dedicação Exclusiva,Arte - Teatro,13/03/2025


--------------------------------------------------


In [11]:
print("DataFrame final pronto para BI:")
print(df_servidores_bi.info())
print("-" * 50)

DataFrame final pronto para BI:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2972 entries, 0 to 2971
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Nome_do_Servidor        2972 non-null   object
 1   Matricula               2972 non-null   object
 2   Vinculo_de_Carreira     2972 non-null   object
 3   Campus_do_IFRN          2972 non-null   object
 4   Setor                   2972 non-null   object
 5   Funcao                  2972 non-null   object
 6   Cargo                   2972 non-null   object
 7   Jornada_de_Trabalho     2972 non-null   object
 8   Disciplina_de_Ingresso  2972 non-null   object
 9   Ano                     2972 non-null   object
dtypes: object(10)
memory usage: 232.3+ KB
None
--------------------------------------------------


In [12]:
# @title --- 4. Exportação para o BigQuery ---

# --- Configurações de Destino do BigQuery ---
project_id = "pnp-data-extraction" # Substitua pelo ID do seu projeto
dataset_id = "pnp_dados_IFRN"      # Nome do conjunto de dados que você já criou
table_id = "df_servidores_ifrn"       # Nome da tabela que será criada ou substituída
destination_table = f"{dataset_id}.{table_id}"

# --- Definição do Esquema da Tabela (Opcional, mas recomendado) ---
# Com base na saída do df.info(), todas as colunas são objetos (strings).
table_schema = [
    {'name': 'Nome_do_Servidor', 'type': 'STRING'},
    {'name': 'Matricula', 'type': 'STRING'},
    {'name': 'Vinculo_de_Carreira', 'type': 'STRING'},
    {'name': 'Campus_do_IFRN', 'type': 'STRING'},
    {'name': 'Setor', 'type': 'STRING'},
    {'name': 'Funcao', 'type': 'STRING'},
    {'name': 'Cargo', 'type': 'STRING'},
    {'name': 'Jornada_de_Trabalho', 'type': 'STRING'},
    {'name': 'Disciplina_de_Ingresso', 'type': 'STRING'},
    {'name': 'Ano', 'type': 'STRING'}
]

# --- Execução da Exportação ---
# Verifica se o DataFrame final não está vazio
if not df_servidores_bi.empty:
    print(f"Iniciando a exportação de {len(df_servidores_bi):,} linhas para o BigQuery...")
    print(f"Destino: {project_id}.{destination_table}")

    try:
        # Envia o DataFrame para o BigQuery, especificando o esquema.
        pandas_gbq.to_gbq(
            df_servidores_bi,
            destination_table=destination_table,
            project_id=project_id,
            if_exists='replace',
            table_schema=table_schema, # Adicionamos o esquema aqui
            progress_bar=True
        )
        print(f"\n✔ DataFrame exportado com sucesso para o BigQuery!")
        print(f"Link para a tabela: https://console.cloud.google.com/bigquery?project={project_id}&ws=!1m5!1m4!4m3!1s{project_id}!2s{dataset_id}!3s{table_id}")
    except Exception as e:
        print(f"❌ ERRO durante a exportação para o BigQuery: {e}")
else:
    print("⚠️ AVISO: O DataFrame final está vazio. Nenhuma exportação foi realizada.")


Iniciando a exportação de 2,972 linhas para o BigQuery...
Destino: pnp-data-extraction.pnp_dados_IFRN.df_servidores_ifrn


100%|██████████| 1/1 [00:00<00:00, 9238.56it/s]


✔ DataFrame exportado com sucesso para o BigQuery!
Link para a tabela: https://console.cloud.google.com/bigquery?project=pnp-data-extraction&ws=!1m5!1m4!4m3!1spnp-data-extraction!2spnp_dados_IFRN!3sdf_servidores_ifrn
